## Nhập dữ liệu, lấy từ tập dữ liệu label 6,7

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving MNIST_test.csv to MNIST_test.csv
Saving MNIST_train.csv to MNIST_train.csv


In [ ]:
train_data=pd.read_csv('MNIST_train.csv')
test_data=pd.read_csv('MNIST_test.csv')

NameError: name 'pd' is not defined

In [ ]:
train_df = train_data[train_data['label'].isin([7,1])]
test_df = test_data[test_data['label'].isin([7,1])]

NameError: name 'train_data' is not defined

In [ ]:
x_train = train_df.drop(['label'], axis=1)
y_train = train_df['label']

x_test = test_df.drop(['label'], axis=1)
y_test = test_df['label']

## PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(svd_solver='full', n_components=0.86)
pca.fit(x_train)

x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

# Số thành phần chính
print(pca.n_components_)

49


In [ ]:
# Ảnh thứ 14 của thành phần chính thứ 9
print(x_train_pca[2][3])

1119.6633880527706


## Huấn luyện dữ liệu bằng sklearn.ensemble.StackingClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = LogisticRegression(random_state = 23)
svm = SVC(random_state = 23)
fores = RandomForestClassifier(random_state=23)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import StackingClassifier

Class = StackingClassifier(estimators=[
    ('lr', clf),
    ('svm', svm),
    ('sgd', fores)
])

Class.fit(x_train_pca, y_train)
y_pred = Class.predict(x_test_pca)

print(classification_report(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)
conf_mat = confusion_matrix(y_test, y_pred)

## In ra tỉ lệ đúng và ma trận
print(accuracy)
print(conf_mat)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       980
           1       1.00      1.00      1.00      1135

    accuracy                           1.00      2115
   macro avg       1.00      1.00      1.00      2115
weighted avg       1.00      1.00      1.00      2115

0.9990543735224586
[[ 978    2]
 [   0 1135]]


Từ ma trận:<br>
[0,0] -> tn<br>
[0,1] -> fp<br>
[1,0] -> fn<br>
[1,1] -> tp<br>

## Huấn luyện bằng sklearn.ensemble.VotingClassifier



In [ ]:
from sklearn.linear_model import SGDClassifier

clf = LogisticRegression(solver='liblinear', random_state=11)
svm = SVC(random_state = 11)
sgd = SGDClassifier(random_state=11)

In [ ]:
from sklearn.ensemble import VotingClassifier

Voting_class = VotingClassifier(estimators=[
    ('lr', clf),
    ('svm', svm),
    ('sgd', sgd)
], voting='hard')

Voting_class.fit(x_train_pca, y_train)
y_pred = Voting_class.predict(x_test_pca)

print(classification_report(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)
conf_mat = confusion_matrix(y_test, y_pred)

print(accuracy)
print(conf_mat)

              precision    recall  f1-score   support

           6       1.00      1.00      1.00       958
           7       1.00      1.00      1.00      1028

    accuracy                           1.00      1986
   macro avg       1.00      1.00      1.00      1986
weighted avg       1.00      1.00      1.00      1986

0.9964753272910373
[[ 954    4]
 [   3 1025]]
